In [10]:
!pip install torch===1.11.0 torchvision===0.12.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached torch-1.11.0-cp310-cp310-manylinux1_x86_64.whl (750.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 59.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1+cu118
    Uninstalling torchvision-0.15.1+cu118:
      Successfully uninstalled torchvision-0.15.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.15.1 requires torch==2.0.0, but you have torch 1.11.0 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have tor

# RBM referd from: https://github.com/devalindey/Recommender-System-with-RBM-using-PyTorch

# KGCN refered from: https://github.com/hwwang55/KGCN

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable


In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable


In [47]:
#training_set.head()

In [100]:
data=pd.read_csv('final.csv', header=None)

In [101]:
data.head()

,0,1,2,3,4,5,6
0,76561197970982479,1250,1,76561197970982479,10006,0,0
1,76561197970982479,22200,1,76561197970982479,271,0,0
2,76561197970982479,43110,1,76561197970982479,834,0,0
3,js41637,227300,1,76561198035864385,551,0,1
4,js41637,239030,1,76561198035864385,349,0,1


In [102]:
data.rename(columns={6:'userId',1:'itemId',2:'rating'},inplace = True)

In [103]:
data.head()

,0,itemId,rating,3,4,5,userId
0,76561197970982479,1250,1,76561197970982479,10006,0,0
1,76561197970982479,22200,1,76561197970982479,271,0,0
2,76561197970982479,43110,1,76561197970982479,834,0,0
3,js41637,227300,1,76561198035864385,551,0,1
4,js41637,239030,1,76561198035864385,349,0,1


In [82]:
data1=data[['id','item_id','recommend']]

In [104]:
data1=data.copy()

In [105]:
data1.head()

,0,itemId,rating,3,4,5,userId
0,76561197970982479,1250,1,76561197970982479,10006,0,0
1,76561197970982479,22200,1,76561197970982479,271,0,0
2,76561197970982479,43110,1,76561197970982479,834,0,0
3,js41637,227300,1,76561198035864385,551,0,1
4,js41637,239030,1,76561198035864385,349,0,1


In [106]:
data1.shape

(40315, 7)

In [108]:
data1.userId.value_counts()

5177     40
2073     36
2980     36
2407     36
6570     36
         ..
10161     1
10163     1
10165     1
10166     1
2784      1
Name: userId, Length: 18741, dtype: int64

In [109]:
#keeping those data that has users that rated more than 20 games

x = data1.groupby('userId').count()['rating'] >= 30

In [110]:
x.shape

(18741,)

In [111]:
educated_users = x[x].index

In [112]:
filtered_rating = data1[data1['userId'].isin(educated_users)]

In [131]:
filtered_rating.shape

(216, 7)

In [20]:
data1.loc[data1['recommend'] ==True, 'recommend'] = 1

C:\Users\OMKAR\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [21]:
data1.loc[data1['recommend'] ==False, 'recommend'] = 0

In [22]:
data1.head()

,id,item_id,recommend
0,0,1250,1
1,0,22200,1
2,0,43110,1
3,1,227300,1
4,1,239030,1


In [114]:
filtered_rating.dtypes

0         object
itemId     int64
rating     int64
3          int64
4          int64
5          int64
userId     int64
dtype: object

In [132]:
data2=filtered_rating[['itemId','rating','userId']]

In [24]:
data1['recommend']=data1['recommend'].astype('int')

C:\Users\OMKAR\AppData\Local\Temp\ipykernel_10484\1781703612.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['recommend']=data1['recommend'].astype('int')


In [133]:
from sklearn.model_selection import train_test_split

training_set, test_set = train_test_split(data2, train_size=0.75)

In [53]:
pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import pandas as pd
import numpy as np
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly
from surprise import KNNBasic
from surprise import KNNBaseline
from surprise import SVD
from surprise import SVDpp
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise import KNNWithMeans

In [125]:
reader = Reader(line_format = 'user item rating', rating_scale=(1,5))

data2 = Dataset.load_from_df(data2, reader)

In [30]:
type(data2)

surprise.dataset.DatasetAutoFolds

In [134]:
traindata, testdata = train_test_split(data2, test_size= 0.25)

In [38]:
training_set = np.array(traindata)
test_set = np.array(testdata)

In [135]:
training_set = np.array(training_set, dtype = 'int')

test_set = np.array(test_set, dtype = 'int')


In [137]:
#training_set

In [ ]:
# Passing to their algorithm refered from github RBM: 

In [75]:
#training_set

In [138]:
# Getting the number of users and items
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_items = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [140]:
#nb_users,nb_items

In [141]:
# Converting the data into an array with users in lines and items in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_items = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_items)
        ratings[id_items - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)


IndexError: index -2 is out of bounds for axis 0 with size 1

In [60]:
traindata

In [16]:
type(test_set)

list

In [18]:
len(test_set[0])

1682

In [ ]:
nb_users

943

In [ ]:
nb_items

1682

In [128]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [127]:
# Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0


# Creating the architecture of the Neural Network

class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)   #100*1682                 
        self.a = torch.randn(1, nh)    #1*100                       
        self.b = torch.randn(1, nv)    #1*1682                   
    def sample_h(self, x):                                     
        wx = torch.mm(x, self.W.t())    #100*1682 * 1682*100 = 100*100
        activation = wx + self.a.expand_as(wx)                  
        p_h_given_v = torch.sigmoid(activation)     #100*100
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):                                      
        wy = torch.mm(y, self.W)    #100*100 * 100*1682 = 100*1682
        activation = wy + self.b.expand_as(wy)                  
        p_v_given_h = torch.sigmoid(activation)     #100*1682
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):                         
        self.W += (torch.mm(v0.t(), ph0)- torch.mm(vk.t(), phk)).t()    #100*1682 
                             #!= 1682*100 * 100*100 - 1682*100 * 100*100 = 1682*100
        self.b += torch.sum((v0-vk), 0)                            
        self.a += torch.sum((ph0-phk), 0)  

In [129]:
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)  
print(rbm.W.size())

torch.Size([100, 3])


In [130]:
# Training the RBM
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user + batch_size]
        v0 = training_set[id_user:id_user + batch_size]
        ph0, _ = rbm.sample_h(v0)
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            vk[v0 < 0] = v0[v0 < 0]
        phk, _ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        s += 1.
    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss / s))


epoch: 1 loss: tensor(nan)
epoch: 2 loss: tensor(nan)
epoch: 3 loss: tensor(nan)
epoch: 4 loss: tensor(nan)
epoch: 5 loss: tensor(nan)
epoch: 6 loss: tensor(nan)
epoch: 7 loss: tensor(nan)
epoch: 8 loss: tensor(nan)
epoch: 9 loss: tensor(nan)
epoch: 10 loss: tensor(nan)


In [142]:
# Testing the RBM
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

TypeError: mm(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [ ]:
#########################

# NCF without meta data

# Reference: https://github.com/johnwang21/TA-Neural-Collaborative-Filtering

In [43]:
final_new['rating']=ratingrandom

In [1]:
import pandas as pd

In [84]:
final_new=pd.read_csv('final.csv',header=None)

In [87]:
final_new.rename(columns = {1:'item_id',2:'rating',6:'user_id'}, inplace = True)

In [88]:
final_new.head()

,0,item_id,rating,3,4,5,user_id
0,76561197970982479,1250,1,76561197970982479,10006,0,0
1,76561197970982479,22200,1,76561197970982479,271,0,0
2,76561197970982479,43110,1,76561197970982479,834,0,0
3,js41637,227300,1,76561198035864385,551,0,1
4,js41637,239030,1,76561198035864385,349,0,1


In [8]:
final_new=pd.read_csv('user_reviewdatawithrating.csv')

In [9]:
final_new.head()

,user_id,item_id,recommend,playtime_forever,playtime_2weeks,rating
0,76561197970982479,1250,True,10006,0,4.5
1,76561197970982479,22200,True,271,0,3.5
2,76561197970982479,43110,True,834,0,4.0
3,js41637,227300,True,551,0,5.0
4,js41637,239030,True,349,0,2.5


In [2]:

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import keras
from keras import layers 
import tensorflow as tf
from keras.models import Model
from keras.optimizers import Adam, SGD

In [3]:
print("Tensorflow version " + tf.__version__)
np.random.seed(123)
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.12.0
Running on TPU  ['10.21.27.18:8470']


In [91]:
traindata, testdata = train_test_split(final_new, test_size= 0.25)

In [92]:
rating_df = traindata.rename(columns={'user_id':'u_id','item_id':'s_id','rating':'rate'})

test_df = testdata.rename(columns={'user_id':'u_id','item_id':'s_id','rating':'rate'})

In [93]:
rating_df.head()

,0,s_id,rate,3,4,5,u_id
13528,sickbubblez,291550,1,76561198080793429,13556,320,5524
17426,hhaz,241930,1,76561197976643758,1961,0,7157
9048,PepperMinty,312990,1,76561198077272601,121,0,3652
30840,Whyyyyyyyyyyyyyyyy,113400,-1,76561198060934495,3457,0,13500
19507,76561198077248859,730,1,76561198077248859,91019,3119,8093


In [94]:
test_df.head()

,0,s_id,rate,3,4,5,u_id
23231,Joshieboy2811,33930,1,76561198055336474,12003,0,9848
6084,oldsauce,268750,1,76561198086972859,2785,78,2417
18419,Bawst,210770,-1,76561198011069017,138,0,7599
730,76561198057521157,251570,1,76561198057521157,5167,0,331
29247,76561198081966701,224260,1,76561198081966701,1188,9,12701


In [95]:
rating_df.dtypes


0       object
s_id     int64
rate     int64
3        int64
4        int64
5        int64
u_id     int64
dtype: object

In [23]:
rating_df['s_id']=rating_df['s_id'].astype('object')

In [96]:
rating_df = rating_df.sample(frac=1, random_state=73)
rating_df.s_id = pd.Categorical(rating_df.s_id)
rating_df['s_id'] = rating_df.s_id.cat.codes
print(rating_df.dtypes)
X = rating_df[['u_id', 's_id']].values #,'Frequency','User_Deg','Channel_Deg'
y = rating_df["rate"]

test_df = test_df.sample(frac=1, random_state=73)
test_df.s_id = pd.Categorical(test_df.s_id)
test_df['s_id'] = test_df.s_id.cat.codes
test_df.dtypes
test_in_u = test_df['u_id'].values #,'Frequency','User_Deg','Channel_Deg'
test_in_s = test_df['s_id'].values
test_rating = test_df["rate"]

0       object
s_id     int16
rate     int64
3        int64
4        int64
5        int64
u_id     int64
dtype: object


In [105]:
np

array([ 77, 213, 417, ..., 376, 387, 385], dtype=int16)

In [97]:
test_set_size = 5000 #10k for test set
train_indices = rating_df.shape[0] - test_set_size 

X_train, X_val, y_train, y_val = (
    X[:train_indices],
    X[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

print('> Train set ratings: {}'.format(len(y_train)))
print('> Test set ratings: {}'.format(len(y_val)))

> Train set ratings: 25236
> Test set ratings: 5000


In [98]:
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_val_array = [X_val[:, 0], X_val[:, 1]]

In [99]:
in_u_dim = len(rating_df["u_id"].unique())
in_s_dim = len(rating_df["s_id"].unique())
 
latent_out_dim = 8 #embedding size

In [100]:
from keras.layers import Add, Activation, Lambda, BatchNormalization, Concatenate, Dropout, Input, Embedding, Dot, Reshape, Dense, Flatten

def ncf():
    
    user_gmf = Input(name = 'u_in_gmf', shape = [1])
    game_gmf = Input(name = 's_in_gmf', shape = [1])

    user_embedding_gmf = Embedding(name = 'u_emb_gmf', input_dim = in_u_dim, output_dim = latent_out_dim)(user_gmf)
    game_embedding_gmf = Embedding(name = 's_emb_gmf', input_dim = in_s_dim, output_dim = latent_out_dim)(stream_gmf)
    
    x = tf.keras.layers.Multiply()([user_embedding_gmf, stream_embedding_gmf])
    x = Model(inputs=[user_gmf,stream_gmf], outputs=x)


    user_mlp = Input(name = 'u_in_mlp', shape = [1])
    stream_mlp = Input(name = 's_in_mlp', shape = [1])

    user_embedding_mlp = Embedding(name = 'u_emb_mlp', input_dim = in_u_dim,output_dim = latent_out_dim)(user_mlp)
    game_embedding_mlp = Embedding(name = 's_emb_mlp', input_dim = in_s_dim, output_dim = latent_out_dim)(stream_mlp)

    y = tf.keras.layers.Concatenate()([user_embedding_mlp, stream_embedding_mlp])
    y = Flatten()(y)

    y = keras.layers.Dense(1, activation='relu')(y) 
    y = keras.layers.Dense(2, activation='relu')(y) 
    y = keras.layers.Dense(3, activation='relu')(y) 
    y = keras.layers.Dense(4, activation='relu')(y) 
    y = keras.layers.Dense(5, activation='relu')(y) 
    y = keras.layers.Dense(6, activation='relu')(y) 
    y = keras.layers.Dense(7, activation='relu')(y) 
    y = keras.layers.Dense(8, activation='relu')(y) 
    y = keras.layers.Reshape([1, 8])(y)
    y = Model(inputs=[user_mlp,stream_mlp], outputs=y)


    #combined https://pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/
    combined = tf.keras.layers.Concatenate()([x.output, y.output])
    z = Dense(1, kernel_initializer='lecun_uniform')(combined)
    z = Activation("sigmoid")(z)
  
    model = Model(inputs=[[user_gmf,stream_gmf], [user_mlp,stream_mlp]], outputs=z)

    model.compile(optimizer='sgd', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
      #https://www.tensorflow.org/api_docs/python/tf/keras/metrics/RootMeanSquaredError
    
    return model

with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  model = ncf()
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 u_in_mlp (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 s_in_mlp (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 u_emb_mlp (Embedding)          (None, 1, 8)         127336      ['u_in_mlp[0][0]']               
                                                                                                  
 s_emb_mlp (Embedding)          (None, 1, 8)         7160        ['s_in_mlp[0][0]']               
                                                                                           

In [103]:
batch_size = 100
history = model.fit(
    x=[X_train_array,X_train_array],
    y=[y_train,y_train],
    batch_size=batch_size,
    epochs=50,
    verbose=1,
    validation_data=([X_val_array, X_val_array], [y_val, y_val])
)


Epoch 1/50
253/253 [==============================] - 12s 39ms/step - loss: 0.4163 - root_mean_squared_error: 0.6452 - val_loss: 0.4132 - val_root_mean_squared_error: 0.6428
Epoch 2/50
253/253 [==============================] - 9s 35ms/step - loss: 0.3894 - root_mean_squared_error: 0.6240 - val_loss: 0.3960 - val_root_mean_squared_error: 0.6293
Epoch 3/50
253/253 [==============================] - 7s 28ms/step - loss: 0.3762 - root_mean_squared_error: 0.6133 - val_loss: 0.3877 - val_root_mean_squared_error: 0.6226
Epoch 4/50
253/253 [==============================] - 7s 28ms/step - loss: 0.3694 - root_mean_squared_error: 0.6078 - val_loss: 0.3834 - val_root_mean_squared_error: 0.6192
Epoch 5/50
253/253 [==============================] - 7s 29ms/step - loss: 0.3658 - root_mean_squared_error: 0.6048 - val_loss: 0.3812 - val_root_mean_squared_error: 0.6175
Epoch 6/50
253/253 [==============================] - 7s 28ms/step - loss: 0.3638 - root_mean_squared_error: 0.6031 - val_loss: 0.3801

In [104]:
test_pred = model.predict_on_batch([[test_in_u, test_in_s],[test_in_u, test_in_s]])
test_pred

array([[[0.7990857 ]],

       [[0.7989429 ]],

       [[0.7989198 ]],

       ...,

       [[0.7991487 ]],

       [[0.79901814]],

       [[0.7990792 ]]], dtype=float32)

In [65]:
type(test_pred)

numpy.ndarray

In [66]:
test_pred.size

10079

In [67]:
list_1 = test_pred.tolist()

In [72]:
flat_list = [item for l in list_1 for item in list_1]

101586241

In [1]:
#flat_list.sort(key=lambda x: x[1], reverse=True)


In [57]:
from sklearn.metrics import mean_squared_error

#rms = mean_squared_error(y_actual, y_predicted, squared=False)
rms = mean_squared_error(test_rating.to_numpy(), test_pred.reshape((len(test_pred),)), squared=False)
rms

2.516780242021213

In [62]:
#predictions = [[row.userID, row.itemID, model.predict(row.test_df['u_id'], row.test_df['s_id'])]
 #              for (_, row) in test_df.iterrows()]

In [78]:
test_in_u = test_df['u_id'].values #,'Frequency','User_Deg','Channel_Deg'
test_in_s = test_df['s_id'].values
test_rating = test_df["rate"]

In [81]:
#model.predict([[17096, 218620],[17096, 218620]])

In [144]:
#model.predict_on_batch([[np.ndarray(np.int64(0)), np.ndarray(np.int64(10))],[np.ndarray(np.int64(0)), np.ndarray(np.int64(10))]])

In [109]:
type(test_in_u[0])

numpy.int64

In [110]:
np.int64(10)

10

In [119]:
type(test_in_u)

numpy.ndarray

In [128]:
test_in_u.shape

(10079,)

In [130]:
test_in_u

(10,)

In [143]:
user_tmp = np.ndarray(1)
item_tmp = np.ndarray(1)

In [144]:
user_tmp[0] = 0
item_tmp[0] = 10
#item_tmp[1] = 11

In [145]:
model.predict_on_batch([[user_tmp, item_tmp], [user_tmp, item_tmp]])

array([[[0.79934275]]], dtype=float32)

# NCF and other library to use : https://towardsdatascience.com/neural-collaborative-filtering-96cef1009401

# using library: https://pypi.org/project/recommenders/

# using https://github.com/microsoft/recommenders



In [ ]:
[['u_id', 's_id']].values #,'Frequency','User_Deg','Channel_Deg'
y = rating_df["rate"]

In [148]:
pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [149]:
pip install recommender-utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 3.8 MB/s eta 0:00:00


In [164]:
# importing required libraries

import sys
sys.path.append("../../")
import time
import os
import shutil
# import papermill as pm
import pandas as pd
import numpy as np
import tensorflow as tf
from reco_utils.recommender.ncf.ncf_singlenode import NCF
from reco_utils.recommender.ncf.dataset import Dataset as NCFDataset
from reco_utils.dataset.python_splitters import python_chrono_split
from reco_utils.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)
from reco_utils.common.constants import SEED as DEFAULT_SEED

# top k items to recommend
TOP_K = 10



100%|██████████| 4.81k/4.81k [00:00<00:00, 26.5kKB/s]
<ipython-input-164-429520df2ef6>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'item_id':'itemID','user_id':'userID'}, inplace = True)


In [165]:
df2.dtypes

userID         int64
itemID         int64
rating       float64
timestamp      int64
dtype: object

In [167]:
#final_new
df['timestamp'] = 0
df

<ipython-input-167-4b9a7cf19301>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamp'] = 0


,userID,itemID,rating,timestamp
0,0,1250,1,0
1,0,22200,1,0
2,0,43110,1,0
3,1,227300,1,0
4,1,239030,1,0
...,...,...,...,...
40310,18739,730,1,0
40311,18740,233270,1,0
40312,18740,130,1,0
40313,18740,70,1,0


In [178]:
df.dtypes

userID         int64
itemID         int64
rating       float64
timestamp      int64
dtype: object

In [169]:
df['rating']=df['rating'].astype('float64')

<ipython-input-169-6e04a6cde8f6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating']=df['rating'].astype('float64')


In [174]:
tf.compat.v1.disable_eager_execution()



In [176]:
tf.contrib=tf.compat.v1.estimator


In [177]:
# chronological spilt on every user
train, test = python_chrono_split(df, 0.75)
#train, test = train_test_split(df2, test_size= 0.25)
# preparing the data
# This function by default takes sampling on unobserved entries to account for negative feedback 
# It only takes rows which atleast have user id, item id and rating
# I know we discussed the benifits of using implicit data but NCF works for explicit data too 
# This data contains rating and hence is the textbook definition of explicit data
data = NCFDataset(train=train, test=test, seed=SEED)

model = NCF(n_users=data.n_users,n_items=data.n_items,model_type="NeuMF",n_factors=4,layer_sizes=[16,8,4],\
            n_epochs=EPOCHS,batch_size=BATCH_SIZE,learning_rate=1e-3,verbose=10,seed=SEED)
# n_factors (int): Dimension of latent space.
# layer_sizes (list): Number of layers for MLP.

# training the model
model.fit(data)

predictions = [[row.userID, row.itemID, model.predict(row.userID, row.itemID)]
               for (_, row) in test.iterrows()]

# saving the predictions in a dataframe
predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])
predictions.head()

/usr/local/lib/python3.10/dist-packages/reco_utils/recommender/ncf/dataset.py:79: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train if test is None else train.append(test)


AttributeError: ignored

In [ ]:
tf.estimator.inputs.numpy_input_fn

In [ ]:
|

# SMOTE

In [145]:
data1.head()

,0,itemId,rating,3,4,5,userId
0,76561197970982479,1250,1,76561197970982479,10006,0,0
1,76561197970982479,22200,1,76561197970982479,271,0,0
2,76561197970982479,43110,1,76561197970982479,834,0,0
3,js41637,227300,1,76561198035864385,551,0,1
4,js41637,239030,1,76561198035864385,349,0,1


In [146]:
data2=data1[['userId','itemId','rating']]

In [147]:
X_train=data2[['userId','itemId']]

y_train=data2[['rating']]

In [148]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [149]:
X_res

,userId,itemId
0,0,1250
1,0,22200
2,0,43110
3,1,227300
4,1,239030
...,...,...
72369,7369,341822
72370,17384,301520
72371,8275,215618
72372,4445,221100


In [150]:
y_res

,rating
0,1
1,1
2,1
3,1
4,1
...,...
72369,-1
72370,-1
72371,-1
72372,-1


In [151]:
# Not correct way as its creating new user with other item which would disrupt the prediction 